In [74]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from PIL import Image

In [84]:
ab = pd.read_csv('abortion_suvel.csv', thousands = ',')

In [85]:
ab.head()

,State,Status_detail,Status,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Alabama,Banned,1,10280,9523,9076,8485,8080,5899,6642,6063,6484,6009
1,Alaska,Legal,0,1715,1627,1632,1450,1518,1334,1260,1255,1283,1270
2,Arizona,Threatened,1,11438,14401,13340,13401,12900,12655,13332,12533,12438,13097
3,Arkansas,Banned,1,4532,4033,3782,3730,4253,3771,3207,3249,3069,2963
4,California**,Legal,0,0,0,0,0,0,0,0,0,0,0


In [86]:
ab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   State          52 non-null     object
 1   Status_detail  52 non-null     object
 2   Status         52 non-null     int64 
 3   2010           52 non-null     int64 
 4   2011           52 non-null     int64 
 5   2012           52 non-null     int64 
 6   2013           52 non-null     int64 
 7   2014           52 non-null     int64 
 8   2015           52 non-null     int64 
 9   2016           52 non-null     int64 
 10  2017           52 non-null     int64 
 11  2018           52 non-null     int64 
 12  2019           52 non-null     int64 
dtypes: int64(11), object(2)
memory usage: 4.9+ KB


In [108]:
data = ab[['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']].to_numpy()
target = ab['Status'].to_numpy()

In [140]:
ab1 = pd.read_csv('abortion_suvel_2.csv', thousands = ',')

In [ ]:
# data = ab1[['Alabama','Alaska','Arizona','Arkansas','California**','Colorado','Connecticut','Delaware','District of Columbia','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland**','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire**','New Jersey','New Mexico','New York City','New York State','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']].to_numpy()
# target = ab1['Status'].to_numpy()

In [109]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [110]:
print(train_input.shape, test_input.shape)

(41, 10) (11, 10)


In [111]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [112]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.6097560975609756
0.6363636363636364


In [113]:
print(lr.coef_, lr.intercept_)

[[-0.26317556 -0.28953747 -0.05747408  0.08574162 -0.2006979   0.01424491
   0.04114338  0.10137303  0.15159965  0.8046724 ]] [0.27172629]


In [114]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_input, train_target)
print("no-prune training score: ", dt.score(train_input, train_target))
print("no-prune testing score: ",dt.score(test_input, test_target))


no-prune training score:  1.0
no-prune testing score:  0.45454545454545453


In [115]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(train_input, train_target)
print("prune training score: ", dt.score(train_input, train_target))
print("prune testing score: ",dt.score(test_input, test_target))

prune training score:  0.8048780487804879
prune testing score:  0.5454545454545454


In [141]:
# import matplotlib.pyplot as plt
# from sklearn.tree import plot_tree
# plt.figure(figsize=(20,7))
# plot_tree(dt, filled=True, feature_names=['2019', '2018'], max_depth=3)
# plt.show()

In [119]:
print(dt.feature_importances_)

[0.19663656 0.         0.16568725 0.         0.         0.
 0.         0.         0.         0.63767619]


In [120]:
# 5 KFold cross validation
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.6305555555555555


In [121]:
# 10 KFold cross validation
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.5549999999999999


In [123]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.001, 0.002, 0.003, 0.004, 0.005]}

In [124]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [125]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.001, 0.002, 0.003, 0.004,
                                                   0.005]})

In [126]:
dt = gs.best_estimator_
print("post-prune from training score: ", dt.score(train_input, train_target))

post-prune from training score:  1.0


In [127]:
print(gs.best_estimator_)

DecisionTreeClassifier(min_impurity_decrease=0.001, random_state=42)


In [128]:
print(gs.cv_results_['mean_test_score'])

[0.58333333 0.58333333 0.58333333 0.58333333 0.58333333]


In [129]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.001}


In [130]:
from scipy.stats import uniform, randint

In [131]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1,25),
         }

In [132]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0xb7052c0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0xb708550>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0xb6fb1b8>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0xae7fcb0>},
                   random_state=42)

In [133]:
print(gs.best_params_)

{'max_depth': 31, 'min_impurity_decrease': 0.0006142344384136117, 'min_samples_leaf': 3, 'min_samples_split': 6}


In [134]:
print("post-prune from validation: ", np.max(gs.cv_results_['mean_test_score']))

post-prune from validation:  0.6305555555555555


In [135]:
dt = gs.best_estimator_
print("post-prune from testing: ", dt.score(test_input, test_target))

post-prune from testing:  0.36363636363636365
